Clone `onnx/models` and then pull all ONNX models to some folder (~50GB at the time of writing)

`git clone https://github.com/onnx/models.git`

`git lfs pull --include="*.onnx" --exclude=""`


In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib
import onnx
import matplotlib.pyplot as plt

In [2]:
PATH = Path("C:/ONNX/models")

In [3]:
models = []

for area in ["text", "vision"]:
    for problem in os.scandir(PATH / area):
        for model in os.scandir(PATH / area / problem):
            for root, _, files in os.walk(PATH / area / problem / model):
                for file in files:
                    if file.endswith('.onnx'):
                        file_path = os.path.join(root, file)
                        models.append({
                            "area": area,
                            "problem": problem.name,
                            "model": model.name,
                            "version": file,
                            "size": os.path.getsize(file_path),
                            "path": file_path
                        })
        
models = pd.DataFrame(models)

In [4]:
model_ops = []

for index, row in tqdm(models.iterrows()):
    model = onnx.load(row["path"])
    ops_present = set()

    for node in model.graph.node:
        ops_present.add(node.op_type)
    
    model_ops.append(ops_present)

models["ops"] = model_ops

168it [00:37,  4.44it/s]


In [5]:
models.sort_values("size", ascending=False)
models

,area,problem,model,version,size,path,ops
0,text,machine_comprehension,bert-squad,bertsquad-10.onnx,435852734,C:\ONNX\models\text\machine_comprehension\bert...,"{ConstantOfShape, Tanh, Sub, Softmax, Mul, Cas..."
1,text,machine_comprehension,bert-squad,bertsquad-12-int8.onnx,124565601,C:\ONNX\models\text\machine_comprehension\bert...,"{ConstantOfShape, Tanh, Sub, Softmax, Mul, Cas..."
2,text,machine_comprehension,bert-squad,bertsquad-12.onnx,435852736,C:\ONNX\models\text\machine_comprehension\bert...,"{ConstantOfShape, Tanh, Sub, Softmax, Mul, Cas..."
3,text,machine_comprehension,bert-squad,bertsquad-8.onnx,435882893,C:\ONNX\models\text\machine_comprehension\bert...,"{Tanh, Sub, Softmax, Tile, Mul, Cast, Gather, ..."
4,text,machine_comprehension,bidirectional_attention_flow,bidaf-9.onnx,43522228,C:\ONNX\models\text\machine_comprehension\bidi...,"{ConstantOfShape, Relu, CategoryMapper, Sub, S..."
...,...,...,...,...,...,...,...
163,vision,style_transfer,fast_neural_style,rain-princess-8.onnx,6726529,C:\ONNX\models\vision\style_transfer\fast_neur...,"{Add, Relu, Upsample, Conv, Pad, InstanceNorma..."
164,vision,style_transfer,fast_neural_style,rain-princess-9.onnx,6728029,C:\ONNX\models\vision\style_transfer\fast_neur...,"{Shape, Add, Cast, Div, Relu, Constant, Gather..."
165,vision,style_transfer,fast_neural_style,udnie-8.onnx,6726529,C:\ONNX\models\vision\style_transfer\fast_neur...,"{Add, Relu, Upsample, Conv, Pad, InstanceNorma..."
166,vision,style_transfer,fast_neural_style,udnie-9.onnx,6728029,C:\ONNX\models\vision\style_transfer\fast_neur...,"{Shape, Add, Cast, Div, Relu, Constant, Gather..."


In [6]:
models.to_pickle("models-df.pkl")